# papeles package - institutions network analysis example

In this notebook, all institutions from Neurips papers are extracted and then an institutions network is created using the co-location of such institutions in the research papers. Data was obtained using the [neurips_crawler](https://github.com/glhuilli/neurips_crawler).

The `papeles` package is used to extract a clean version of the institutions by processing the front page of the research papers, identifying from there which institutions were involved in the research. Results shows that...



In [15]:
import os
import json
from collections import defaultdict

from tqdm.notebook import tqdm


from papeles.paper.neurips import get_key


## Loading the data

In [16]:
# These are files with encoding issues that were not parse correctly by the pdf_parser 
SKIP_FILES = [
    '5049-nonparametric-multi-group-membership-model-for-dynamic-networks.pdf_headers.txt',
    '4984-cluster-trees-on-manifolds.pdf_headers.txt',
    '5820-alternating-minimization-for-regression-problems-with-vector-valued-outputs.pdf_headers.txt',
    '9065-visualizing-and-measuring-the-geometry-of-bert.pdf_headers.txt'
    '4130-implicit-encoding-of-prior-probabilities-in-optimal-neural-populations.pdf_headers.txt',
    '7118-local-aggregative-games.pdf_headers.txt'
]

NEURIPS_ANALYSIS_DATA_PATH = '/var/data/neurips_analysis'

file_lines = defaultdict(list)
for filename in tqdm(os.listdir(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_headers/')), 'loading files'):
    if filename in SKIP_FILES:
        continue
    with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, './files_headers/', filename), 'r') as f:
        for line in f.readlines():
            file_lines[get_key(filename)].append(line.strip())
            
metadata_path = os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_metadata/')

metadata = {}
for filename in tqdm(os.listdir(metadata_path), 'loading metadata'):
    with open(os.path.join(metadata_path, filename), 'r') as f: # open in readonly mode
        for line in f.readlines():
            data = json.loads(line)
            metadata[get_key(data['pdf_name'])] = data


In [17]:
from papeles.paper.neurips import institutions

from collections import Counter 
import itertools


def _get_file_institutions(lines, filtered_institutions=None):
    file_institutions = []
    for institution in institutions.parse_institutions(lines):
        if institution:
            name = institutions.fix_typo(institution[-1])
            names = institutions.fix_institution_parsing(name)
            for name in names:
                fixed_name = institutions.fix_institution_name(name)
                if filtered_institutions:
                    if fixed_name in filtered_institutions:
                        file_institutions.append(fixed_name)
                else:
                    file_institutions.append(fixed_name)                    
    return file_institutions


In [18]:
inst_counter = Counter()
for file, lines in list(file_lines.items()):
    file_institutions = _get_file_institutions(lines)
    unique_file_institutions = list(set(file_institutions))
    inst_counter.update(unique_file_institutions)

cleaned = len(sorted([x for x in inst_counter.items() if x[1] > 0 and x[0]], key= lambda x: x[1], reverse=True))
total = sum([x[1] for x in inst_counter.items() if x[1] > 0 and x[0]])

print(f'current institutions: {cleaned}')
print(f'total raw institutions: {total}')
print(f'clean-up fraction: {"{:0.2f}".format(1 - cleaned / total)}')

current institutions: 2989
total raw institutions: 9245
clean-up fraction: 0.68


In [26]:
from collections import Counter 
import itertools

import community
import networkx as nx
import matplotlib.pyplot as plt


def build_institutions_graph(file_lines, metadata, inst_counter, freq=None, year=None):
    """
    Build graph using two filters:
    - Frequency that the institution has across all periods of time 
    - Year of publishing 
    """
    filtered_institutions = set([x[0] for x in inst_counter.items() if x[1] > freq and x[0]])
    year_keys = defaultdict(list)
    for k, d in metadata.items():
        year_keys[d.get('year')].append(k)
        
    graph = nx.Graph()
    for file, lines in list(file_lines.items()):
        if year and not (file in year_keys.get(year, {})):
            continue
    
        file_institutions = _get_file_institutions(lines, filtered_institutions)
            
        unique_file_institutions = list(set(file_institutions))
        if len(unique_file_institutions) > 1:
            for i, j in itertools.combinations(unique_file_institutions, 2):
                if graph.has_edge(i, j):
                    graph[i][j]['weight'] += 1
                else:
                    graph.add_edge(i, j, weight=1)
                    graph.add_edge(j, i, weight=1)
        if len(unique_file_institutions) == 1:
            i = unique_file_institutions[0]
            graph.add_edge(i, i, weight=1)
    return graph


def graph_to_d3js(graph):
    """
    Output graph compatible with D3.js network structure
    """
    output_graph = {'nodes': [], 'links': []}    
    nodes = set()
    for e in graph.edges():
        output_graph['links'].append({'source': e[0], 'target': e[1], 'value': graph[e[0]][e[1]]['weight']})
        nodes.add(e[0])
        nodes.add(e[1])
    output_graph['nodes'] = [{'id': n, 'group': 1} for n in list(nodes)]
    with open('neurips_institutions.json', 'w') as f:
        json.dump(output_graph, f)      


def plot_graph(graph):
    node_size = []
    for node, degree in graph.degree():
        if degree < 5:
            node_size.append(10)
        elif 5 <= degree < 10:
            node_size.append(40)
        else: 
            node_size.append(90)
    
    partition = community.best_partition(graph)
    
    edge_colors = []
    edge_width = []
    for i, j in graph.edges():
        if graph[i][j]['weight'] < 3:
            edge_colors.append('gray')
            edge_width.append(0.1)
        elif 3 <= graph[i][j]['weight'] < 10:
                edge_colors.append('black')                
                edge_width.append(1)
        elif 10 <= graph[i][j]['weight'] < 20:
                edge_colors.append('blue')                
                edge_width.append(1)
        else:
            edge_colors.append('red')
            edge_width.append(6)
            
    plt.figure(1, figsize=(120,120)) 
    nx.draw(graph, 
            with_labels=True, 
            node_color=list(partition.values()),  #node_color_map, 
            node_size=node_size, 
            font_size=8, 
            edge_color=edge_colors, 
            width=edge_width, 
            alpha=0.8)
    plt.savefig("institutions_graph.png")



In [32]:
g_all_n2 = build_institutions_graph(file_lines, metadata, inst_counter, freq=2) 
print(nx.info(g_all_n2))


Name: 
Type: Graph
Number of nodes: 312
Number of edges: 1828
Average degree:  11.7179


In [20]:
g_all = build_institutions_graph(file_lines, metadata, inst_counter, freq=10) 
print(nx.info(g_all))


Name: 
Type: Graph
Number of nodes: 110
Number of edges: 1074
Average degree:  19.5273


In [21]:
graphs = {}
for year in range(2009, 2020):
    graphs[year] = build_institutions_graph(file_lines, metadata, inst_counter, freq=10, year=year)
    print(f'\nyear: {year}')
    print(nx.info(graphs[year]))


year: 2009
Name: 
Type: Graph
Number of nodes: 67
Number of edges: 110
Average degree:   3.2836

year: 2010
Name: 
Type: Graph
Number of nodes: 65
Number of edges: 123
Average degree:   3.7846

year: 2011
Name: 
Type: Graph
Number of nodes: 67
Number of edges: 134
Average degree:   4.0000

year: 2012
Name: 
Type: Graph
Number of nodes: 77
Number of edges: 152
Average degree:   3.9481

year: 2013
Name: 
Type: Graph
Number of nodes: 76
Number of edges: 137
Average degree:   3.6053

year: 2014
Name: 
Type: Graph
Number of nodes: 78
Number of edges: 161
Average degree:   4.1282

year: 2015
Name: 
Type: Graph
Number of nodes: 88
Number of edges: 204
Average degree:   4.6364

year: 2016
Name: 
Type: Graph
Number of nodes: 91
Number of edges: 219
Average degree:   4.8132

year: 2017
Name: 
Type: Graph
Number of nodes: 100
Number of edges: 296
Average degree:   5.9200

year: 2018
Name: 
Type: Graph
Number of nodes: 102
Number of edges: 334
Average degree:   6.5490

year: 2019
Name: 
Type: Gra

In [22]:
year_keys = defaultdict(list)
for k, d in metadata.items():
    year_keys[d.get('year')].append(k)

missing = []
for k, lines in file_lines.items():
    if k in year_keys[2018]:
        print('*' * 100)
        print(lines)
        inst = _get_file_institutions(lines)
        if inst:
            print(inst)
#             pass
#             print(f'***** {"--".join(inst)}')
        else:
            pass
#             missing.append(lines)
#             print(lines)
print(len(missing))

****************************************************************************************************
['Variance-Reduced Stochastic Gradient Descent', 'on Streaming Data', 'Ellango Jothimurugesan∗ †', 'Carnegie Mellon University', 'ejothimu@cs.cmu.edu', 'Ashraf Tahmasbi∗ ‡', 'Iowa State University', 'tahmasbi@iastate.edu', 'Phillip B. Gibbons†', 'Carnegie Mellon University', 'gibbons@cs.cmu.edu', 'Srikanta Tirthapura‡', 'Iowa State University', 'snt@iastate.edu']
['carnegie mellon university', 'iowa state university', 'carnegie mellon university', 'iowa state university']
****************************************************************************************************
['Adaptation to Easy Data in Prediction with Limited', 'Advice', 'Tobias Sommer Thune', 'Department of Computer Science', 'University of Copenhagen', 'tobias.thune@di.ku.dk', 'Yevgeny Seldin', 'Department of Computer Science', 'University of Copenhagen', 'seldin@di.ku.dk']
['university of copenhagen', 'university of cop

['school of artiﬁcial intelligence university of chinese academy of sciences']
****************************************************************************************************
['Leveraging the Exact Likelihood of Deep Latent', 'Variable Models', 'Pierre-Alexandre Mattei', 'Department of Computer Science', 'IT University of Copenhagen', 'pima@itu.dk', 'Jes Frellsen', 'Department of Computer Science', 'IT University of Copenhagen', 'jefr@itu.dk']
['it university of copenhagen', 'it university of copenhagen']
****************************************************************************************************
['Flexible Neural Representation for Physics Prediction', 'Damian Mrowca1,', '⇤, Chengxu Zhuang2,', '⇤, Elias Wang3,', '⇤, Nick Haber2,4,5 , Li Fei-Fei1 ,', 'Joshua B. Tenenbaum7,8 , and Daniel L. K. Yamins1,2,6', 'Department of Computer Science1, Psychology2, Electrical Engineering3, Pediatrics4 and', 'Biomedical Data Science5, and Wu Tsai Neurosciences Institute6, Stanford, CA 9

['sequel team inria lille france', 'facebook ai research', 'criteo ai lab']
****************************************************************************************************
['Disconnected Manifold Learning for Generative', 'Adversarial Networks', 'Mahyar Khayatkhoei', 'Department of Computer Science', 'Rutgers University', 'm.khayatkhoei@cs.rutgers.edu', 'Ahmed Elgammal', 'Department of Computer Science', 'Rutgers University', 'elgammal@cs.rutgers.edu', 'Maneesh Singh', 'Verisk Analytics', 'maneesh.singh@verisk.com']
['rutgers university', 'rutgers university']
****************************************************************************************************
['On Misinformation Containment in', 'Online Social Networks', 'Guangmo (Amo) Tong', 'Department of Computer and Information Sciences', 'University of Delaware', 'amotong@udel.edu', 'Weili Wu', 'Department of Computer Science', 'University of Texas at Dallas', 'weiliwu@utdallas.edu', 'Ding-Zhu Du', 'Department of Computer Sci

In [23]:
for k, d in sorted(year_keys.items(), key=lambda x: x[0]):
    print(k, len(d))

2009 261
2010 292
2011 305
2012 368
2013 360
2014 411
2015 403
2016 569
2017 679
2018 1008
2019 1427


In [27]:
plot_graph(g_all)

In [31]:
partition = community.best_partition(g_all)
institution_clusters = defaultdict(list)
for k, p in partition.items():
    institution_clusters[p].append(k)
institution_clusters
    

defaultdict(list,
            {0: ['university of oxford',
              'australian national university',
              'deepmind',
              'universite de montreal',
              'university of cambridge',
              'google brain',
              'university of edinburgh',
              'university of british columbia',
              'alan turing institute',
              'max planck institute for intelligent systems',
              'university college london',
              'rensselaer polytechnic institute',
              'uber',
              'facebook ai research',
              'imperial college london',
              'university of sydney',
              'technische universitat berlin',
              'university of warwick'],
             1: ['cornell university',
              'columbia university',
              'hong kong university of science and technology',
              'princeton university',
              'university of pennsylvania',
              'purdue uni

In [33]:
partition = community.best_partition(g_all_n2)
institution_clusters = defaultdict(list)
for k, p in partition.items():
    institution_clusters[p].append(k)
institution_clusters
    

defaultdict(list,
            {0: ['university of oxford',
              'university of cambridge',
              'university of edinburgh',
              'university of california los angeles',
              'alan turing institute',
              'max planck institute for intelligent systems',
              'city university of hong kong',
              'university college london',
              'korea university',
              'university of bristol',
              'university of virginia',
              'nanyang technological university',
              'imperial college london',
              'national university of defense technology',
              'ku leuven',
              'technische universitat berlin',
              'university of southampton',
              'king’s college london',
              'universidad de chile',
              'university carlos iii in madrid',
              'lancaster university',
              'university of edinburgh uk',
              'eurecom',
  

In [34]:
nx.triangles(g_all_n2)

{'university of oxford': 257,
 'cornell university': 152,
 'australian national university': 7,
 'nicta': 5,
 'university of alberta': 44,
 'iowa state university': 3,
 'carnegie mellon university': 609,
 'deepmind': 199,
 'university of washington': 242,
 'northeastern university': 66,
 'columbia university': 284,
 'stanford university': 457,
 'university of pittsburgh': 2,
 'university of amsterdam': 7,
 'google research': 406,
 'university of california irvine': 23,
 'microsoft research': 642,
 'university of massachusetts amherst': 19,
 'hong kong university of science and technology': 0,
 'inria': 136,
 'ecole centrale paris': 1,
 'university of copenhagen': 15,
 'university of toronto': 165,
 'vmware research': 0,
 'universite de montreal': 41,
 'university of cambridge': 173,
 'university of california berkeley': 641,
 'princeton university': 444,
 'google brain': 306,
 'university of edinburgh': 67,
 'peng cheng laboratory': 0,
 'mcgill university': 6,
 'university of waterloo'

In [38]:
all_cliques = nx.enumerate_all_cliques(g_all_n2)
triad_cliques = [x for x in all_cliques if len(x)==3 ]
len(triad_cliques)

4275

In [55]:
data_for_teams = defaultdict(list)
for node in g_all_n2.nodes():
    for edge in g_all_n2.edges(node):
        if g_all_n2[edge[0]][edge[1]]['weight'] < 2:
            continue
        for e in edge:
            if e != node:
                data_for_teams[node].append(e)

for k, v in data_for_teams.items():
    print(k, len(v))

university of oxford 16
cornell university 14
australian national university 2
nicta 3
university of alberta 7
carnegie mellon university 40
deepmind 15
university of washington 15
columbia university 17
stanford university 30
university of amsterdam 1
google research 23
university of california irvine 2
microsoft research 43
university of massachusetts amherst 4
inria 17
ecole centrale paris 1
university of copenhagen 1
university of toronto 12
vmware research 1
universite de montreal 7
university of cambridge 16
university of california berkeley 35
princeton university 31
google brain 18
university of edinburgh 4
university of british columbia 1
universite paris-saclay 1
politecnico di milano 1
university of texas at austin 20
university of pennsylvania 18
technion - israel institute of technology 10
mit-ibm watson ai lab 1
university of california los angeles 11
new york university 12
toyota technological institute at chicago 8
boston university 2
university of illinois at urbana-ch

In [ ]:
"""
1 -> [2, 3]

2 -> [1, 3, 4]

5 -> [6]

[1, 2, 3, 4]

[5, 6]

"""
for k, v in data_for_teams.items():
    for k2, v in data_for_teams.items():
        if k == k2:
            continue
        if 

In [54]:
# https://stackoverflow.com/questions/10301000/python-connected-components

myGraph = {0: [1,2,3], 1: [], 2: [1], 3: [4,5],4: [3,5], 5: [3,4,7], 6: [8], 7: [],8: [9], 9: []}


def getRoots(aNeigh):
    def findRoot(aNode,aRoot):
        while aNode != aRoot[aNode][0]:
            aNode = aRoot[aNode][0]
        return (aNode,aRoot[aNode][1])
    myRoot = {} 
    for myNode in aNeigh.keys():
        myRoot[myNode] = (myNode,0)  
    for myI in aNeigh: 
        for myJ in aNeigh[myI]: 
            (myRoot_myI,myDepthMyI) = findRoot(myI,myRoot) 
            (myRoot_myJ,myDepthMyJ) = findRoot(myJ,myRoot) 
            if myRoot_myI != myRoot_myJ: 
                myMin = myRoot_myI
                myMax = myRoot_myJ 
                if  myDepthMyI > myDepthMyJ: 
                    myMin = myRoot_myJ
                    myMax = myRoot_myI
                myRoot[myMax] = (myMax,max(myRoot[myMin][1]+1,myRoot[myMax][1]))
                myRoot[myMin] = (myRoot[myMax][0],-1) 
    myToRet = {}
    for myI in aNeigh: 
        if myRoot[myI][0] == myI:
            myToRet[myI] = []
    for myI in aNeigh: 
        myToRet[findRoot(myI,myRoot)[0]].append(myI) 
    return myToRet 


getRoots(graph_for_teams).keys()

dict_keys(['university of edinburgh', 'radboud university nijmegen', 'idsia', 'university of helsinki', 'iit delhi'])

In [38]:
centrality = nx.eigenvector_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.2432),
 ('carnegie mellon university', 0.2383),
 ('university of california berkeley', 0.2298),
 ('mit', 0.2287),
 ('princeton university', 0.1906),
 ('stanford university', 0.1891),
 ('google', 0.1862),
 ('google research', 0.175),
 ('harvard university', 0.1622),
 ('university of pennsylvania', 0.1609),
 ('columbia university', 0.1578),
 ('university of texas at austin', 0.1532),
 ('google brain', 0.1446),
 ('university of washington', 0.1427),
 ('ibm research', 0.1407),
 ('university of oxford', 0.1391),
 ('new york university', 0.1383),
 ('university of california san diego', 0.137),
 ('university college london', 0.1298),
 ('university of michigan', 0.1271),
 ('deepmind', 0.1245),
 ('duke university', 0.1235),
 ('tsinghua university', 0.12),
 ('university of cambridge', 0.1193),
 ('university of california los angeles', 0.1158),
 ('toyota technological institute at chicago', 0.1144),
 ('university of illinois at urbana-champaign', 0.114),
 ('cornell unive

In [41]:
centrality = nx.katz_centrality_numpy(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('inria', 0.2136),
 ('ecole normale superieure', 0.1634),
 ('university of oxford', 0.1406),
 ('eth zürich', 0.1383),
 ('university of cambridge', 0.135),
 ('cnrs', 0.131),
 ('max planck institute for intelligent systems', 0.1309),
 ('deepmind', 0.1248),
 ('facebook ai research', 0.1083),
 ('university of copenhagen', 0.1056),
 ('alan turing institute', 0.1052),
 ('university college london', 0.101),
 ('sorbonne universite', 0.0932),
 ('max planck institute', 0.0928),
 ('ecole polytechnique', 0.0883),
 ('universitat pompeu fabra', 0.0871),
 ('imperial college london', 0.0855),
 ('new york university', 0.0807),
 ('university of edinburgh', 0.0771),
 ('epfl', 0.0768),
 ('google brain', 0.0767),
 ('university of sydney', 0.0765),
 ('mit', 0.0733),
 ('ist austria', 0.0706),
 ('uber', 0.0704),
 ('university of genoa', 0.0696),
 ('leiden university', 0.0674),
 ('australian national university', 0.0654),
 ('jst presto', 0.0653),
 ('university of alberta', 0.0649),
 ('university of warwick', 

In [42]:
centrality = nx.closeness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.5165),
 ('carnegie mellon university', 0.5107),
 ('mit', 0.5107),
 ('university of california berkeley', 0.5071),
 ('stanford university', 0.4911),
 ('princeton university', 0.4743),
 ('google research', 0.4706),
 ('google', 0.4663),
 ('university of oxford', 0.4645),
 ('harvard university', 0.461),
 ('university college london', 0.4604),
 ('columbia university', 0.4574),
 ('new york university', 0.4551),
 ('university of pennsylvania', 0.45),
 ('ibm research', 0.45),
 ('university of cambridge', 0.4489),
 ('deepmind', 0.4483),
 ('google brain', 0.4483),
 ('university of california san diego', 0.4478),
 ('university of washington', 0.4467),
 ('university of texas at austin', 0.4461),
 ('duke university', 0.4434),
 ('inria', 0.4428),
 ('cornell university', 0.439),
 ('university of california los angeles', 0.439),
 ('eth zürich', 0.4364),
 ('university of illinois at urbana-champaign', 0.4348),
 ('tsinghua university', 0.4348),
 ('university of michigan', 0.433

In [43]:
centrality = nx.information_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)

NetworkXError: Graph not connected.

In [44]:
centrality = nx.betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('mit', 0.1453),
 ('microsoft research', 0.1286),
 ('university of california berkeley', 0.1044),
 ('carnegie mellon university', 0.1032),
 ('stanford university', 0.0765),
 ('university college london', 0.0638),
 ('university of oxford', 0.0513),
 ('google', 0.0511),
 ('inria', 0.0439),
 ('columbia university', 0.0382),
 ('university of texas at austin', 0.0363),
 ('princeton university', 0.0354),
 ('university of cambridge', 0.0351),
 ('google research', 0.0346),
 ('cnrs', 0.0336),
 ('university of california los angeles', 0.0333),
 ('new york university', 0.0318),
 ('cornell university', 0.0307),
 ('harvard university', 0.0291),
 ('duke university', 0.0274),
 ('deepmind', 0.026),
 ('ntt communication science laboratories', 0.0257),
 ('ibm research', 0.0246),
 ('university of pennsylvania', 0.0232),
 ('university of illinois at urbana-champaign', 0.0228),
 ('university of alberta', 0.0226),
 ('eth zürich', 0.0225),
 ('rutgers university', 0.0216),
 ('chinese university of hong kong'

In [45]:
centrality = nx.current_flow_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


NetworkXError: Graph not connected.

In [46]:
centrality = nx.communicability_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


/Users/gastonlhuillier/Personal/papeles/.python/papeles/lib/python3.8/site-packages/networkx/algorithms/centrality/subgraph_alg.py:246: RuntimeWarning: invalid value encountered in true_divide
  B = (expA - scipy.linalg.expm(A.A)) / expA


[('australian national university', nan),
 ('nicta', nan),
 ('university of alberta', nan),
 ('cifar', nan),
 ('university of amsterdam', nan),
 ('microsoft research', nan),
 ('university of massachusetts amherst', nan),
 ('ecole centrale paris', nan),
 ('inria', nan),
 ('cornell university', nan),
 ('university of california berkeley', nan),
 ('princeton university', nan),
 ('ebay inc', nan),
 ('stanford university', nan),
 ('allen institute for ai', nan),
 ('university of texas at austin', nan),
 ('university of illinois at urbana-champaign', nan),
 ('openai', nan),
 ('brown university', nan),
 ('bar-ilan university', nan),
 ('harvard university', nan),
 ('university of verona', nan),
 ('indian institute of science', nan),
 ('chalmers university of technology', nan),
 ('toyota technological institute at chicago', nan),
 ('max planck institute for intelligent systems', nan),
 ('alan turing institute', nan),
 ('sun yat-sen university', nan),
 ('chinese university of hong kong', nan),
 

In [47]:
hubs, authorities = nx.hits(graph)

In [48]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in hubs.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t

In [49]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in authorities.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t